# Going from differences in Brier score to expected profit

## Expected profit
My probability is $p$.
Market's probability is $q$.

If you believe the event is more likely than the market does ($p>q$), you can buy contracts.   
Payoff iff event occurs is: $1 - q$   
Payoff iff event does not occur: $-q$


Expected profit is:
$p (1-q) + (1-p) (-q) = p - q$

Reverse holds if ($q>p$). Then you sell contracts and the expected profit is $q - p$.

So in general the expected profit is $|p - q|$

Claim: expected profit is square root of differences in Brier score.
$EP = \sqrt{BS_{market} - BS_{you}}$
if you are well calibrated. A.k.a. your probability $p$ is how often that event really actually happens.

Below is a simulation to see if my derivation has mistakes. --- The simulation does not contradict my derivation.

In [ ]:
import numpy as np

p = 0.824
q = 0.78
size = 1000000
occurences = np.random.binomial(n=1, p=p, size=size)
non_occurences = 1 - occurences
earnings = np.sum(occurences * (1 - q))
earnings += np.sum(non_occurences * (-q))
avg_earnings = earnings / size
avg_earnings

my_bs = np.sum((p - occurences )**2) / size
my_bs

market_bs = np.sum((q - occurences )**2) / size
market_bs

expected_profit_via_bs = np.sqrt(market_bs - my_bs)
print(f"Market BS: {market_bs} \nForecaster BS: {my_bs}")
print(avg_earnings, expected_profit_via_bs)

A larger simulation:   
The true probability of the event will be from a distribution. The market's and the forecaster's probability will be a noisy estimate of the true probability. There will be buy and sell options.
This simulation will not include order books and moving a market's probability. Also no transaction cost or risk of ruin will be modelled.


This simulation shows that the expected_profit from Brier score difference math does not hold. But simulating a bunch of different possibilities shows, that there is a linear relationship between the sqaureroot of the difference in Brier score and the actual average profit even when you're not perfectly calibrated. A linear relationship with roughly b=-0.002 and m=0.97. This changes a bit depending on how badly you and the market are calibrated.. But taking those numbers: when expecting a 7% average profit, you'd make an actual roughly 6.59% profit.

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as stats

def mean_v_to_a_b(mean, v):
    a = mean * v
    b = (1-mean) * v
    return a, b

x = np.linspace(0, 1, 1000)
beta = stats.beta.pdf(x, a=5, b=5)
# plt.plot(x, beta)

In [ ]:


n_bets = 1000000
true_p = np.random.beta(a=1, b=1, size=n_bets) #np.array([0.8] * n_bets)

# beta_concentration is a parameter that specifies the shape of the beta distribution. It's equivalent to having seen beta_concentration many coin flips in the beta-bernoulli Bayesian estimation.
market_beta_concentration = 50
market_a, market_b = mean_v_to_a_b(mean=true_p, v=market_beta_concentration)
market_p = np.random.beta(market_a, market_b)

forecaster_beta_concentration = 100
forecaster_a, forecaster_b = mean_v_to_a_b(mean=true_p, v=forecaster_beta_concentration)
forecaster_p = np.random.beta(forecaster_a, forecaster_b)


# True means forecaster buys, False means forecaster sells/buys "No"
buy_sell = forecaster_p > market_p

resolved_positive = (np.random.binomial(n=1, p=true_p) != 0)

expenses = np.sum(market_p * buy_sell)
expenses += np.sum((1-market_p) * ~buy_sell) 
# rewards = np.sum(resolved_positive[buy_sell]) + np.sum(~resolved_positive[~buy_sell])
rewards = np.sum(resolved_positive == buy_sell)

profit = rewards - expenses
avg_profit = profit / n_bets

best_bs = np.sum((true_p - resolved_positive)**2) / n_bets
forecaster_bs = np.sum((forecaster_p - resolved_positive)**2) / n_bets
market_bs = np.sum((market_p - resolved_positive)**2) / n_bets

print(best_bs, forecaster_bs, market_bs)
expected_profit_via_bs = np.sqrt(market_bs - forecaster_bs)
print(avg_profit, expected_profit_via_bs)

In [ ]:
def profit_exp(a, b):
    n_bets = 1000000
    true_p = np.random.beta(a=a, b=b, size=n_bets) #np.array([0.8] * n_bets)

    # beta_concentration is a parameter that specifies the shape of the beta distribution. It's equivalent to having seen beta_concentration many coin flips in the beta-bernoulli Bayesian estimation.
    market_beta_concentration = 10
    market_a, market_b = mean_v_to_a_b(mean=true_p, v=market_beta_concentration)
    market_p = np.random.beta(market_a, market_b)

    forecaster_beta_concentration = 50
    forecaster_a, forecaster_b = mean_v_to_a_b(mean=true_p, v=forecaster_beta_concentration)
    forecaster_p = np.random.beta(forecaster_a, forecaster_b)


    # True means forecaster buys, False means forecaster sells/buys "No"
    buy_sell = forecaster_p > market_p

    resolved_positive = (np.random.binomial(n=1, p=true_p) != 0)

    expenses = np.sum(market_p * buy_sell)
    expenses += np.sum((1-market_p) * ~buy_sell) 
    # rewards = np.sum(resolved_positive[buy_sell]) + np.sum(~resolved_positive[~buy_sell])
    rewards = np.sum(resolved_positive == buy_sell)

    profit = rewards - expenses
    avg_profit = profit / n_bets

    best_bs = np.sum((true_p - resolved_positive)**2) / n_bets
    forecaster_bs = np.sum((forecaster_p - resolved_positive)**2) / n_bets
    market_bs = np.sum((market_p - resolved_positive)**2) / n_bets

    expected_profit_via_bs = np.sqrt(market_bs - forecaster_bs)
    return avg_profit, expected_profit_via_bs

In [ ]:
avg_profits = []
expected_profits = []
for i in range(200):
    a,b = np.random.uniform(0.00000001,250, size=2)
    
    avg_profit, expected_profit_via_bs = profit_exp(a, b)
    avg_profits.append(avg_profit)
    expected_profits.append(expected_profit_via_bs)

In [ ]:
expected_profits[1]

In [ ]:
plt.scatter(expected_profits, avg_profits)
plt.xlim([0, np.max(expected_profits)+0.005])
plt.ylim([0, np.max(expected_profits)+0.005])

In [ ]:
from numpy.polynomial.polynomial import polyfit
b, m = polyfit(expected_profits, avg_profits, 1)
print(b, m)

In [ ]:
x = np.linspace(0,1,1000)
y = x**2
y[int(0.040662*1000)]

In [ ]:
a,b = mean_v_to_a_b(mean=0.8, v=10)
print(a, b)
x = np.linspace(0, 1, 1000)
beta = stats.beta.pdf(x, a=a, b=b)
plt.plot(x, beta)